<a href="https://colab.research.google.com/github/NaveenR2005/reg-no-2023502006/blob/main/Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Install dependencies (Colab only)
!pip install -q gradio torch torchvision

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import gradio as gr
from PIL import Image
import numpy as np

# Define CNN
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(1, 16, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(32 * 7 * 7, 100),
            nn.ReLU(),
            nn.Linear(100, 10)
        )

    def forward(self, x):
        return self.net(x)

# Load and preprocess data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Train model (quick train)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

model.train()
for batch_idx, (images, labels) in enumerate(train_loader):
    if batch_idx > 200: break  # Limit for fast demo
    images, labels = images.to(device), labels.to(device)
    optimizer.zero_grad()
    output = model(images)
    loss = criterion(output, labels)
    loss.backward()
    optimizer.step()
    if batch_idx % 50 == 0:
        print(f"Batch {batch_idx}, Loss: {loss.item():.4f}")

# Predict function
def predict_digit(img: Image.Image):
    img = img.convert('L').resize((28, 28))
    img_tensor = transforms.ToTensor()(img)
    img_tensor = transforms.Normalize((0.5,), (0.5,))(img_tensor)
    img_tensor = img_tensor.unsqueeze(0).to(device)
    model.eval()
    with torch.no_grad():
        output = model(img_tensor)
        probs = torch.softmax(output, dim=1).cpu().numpy()[0]
    return {str(i): float(probs[i]) for i in range(10)}

# Gradio UI (no source/tool args now)
interface = gr.Interface(
    fn=predict_digit,
    inputs=gr.Image(type="pil", label="Draw a digit (0-9)"),
    outputs=gr.Label(num_top_classes=3),
    title="MNIST Digit Classifier (with Deep Learning)",
    description="Draw a digit. The model will predict it using a CNN trained on MNIST."
)

interface.launch(share=True)


Batch 0, Loss: 2.2973
Batch 50, Loss: 0.4320
Batch 100, Loss: 0.3788
Batch 150, Loss: 0.2959
Batch 200, Loss: 0.2309
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7a459f226277922d61.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
